In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor

from scipy.interpolate import make_interp_spline
from scipy.stats import ks_2samp
from scipy.spatial import ConvexHull
from skimage import measure
import warnings

/tmp/ipykernel_8439/46740767.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [3]:
#name_project = 'project/output_FJSJ_17-01/'               
#name_project = 'project_repartition_v3.0/output_repar_01-01/'     
name_project = 'project_repartition_v3.0/output_repar_v9.1_02-01/'               
name_project

'project_repartition_v3.0/output_repar_v9.1_02-01/'

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v3.0/output_repar_v9.1_02-01/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [6]:
key_subworks = info_basic['key_subworks']
key_subworks

['1--07-03',
 '2--08-03',
 '3--09-03',
 '4--10-03',
 '5--11-03',
 '6--12-03',
 '7--13-03',
 '8--14-03',
 '9--15-03',
 '10--16-03',
 '11--18-03',
 '12--19-03',
 '13--20-03',
 '14--21-03',
 '15--22-03',
 '16--23-03',
 '17--24-03',
 '18--25-03',
 '19--26-03',
 '20--27-03',
 '21--28-03',
 '22--29-03',
 '23--30-03',
 '24--31-03',
 '25--32-03',
 '26--33-03',
 '27--34-03',
 '28--35-03',
 '29--36-03',
 '30--37-03',
 '31--38-03',
 '32--39-03',
 '33--40-03',
 '34--41-03',
 '35--42-03',
 '36--43-03',
 '37--44-03',
 '38--46-03',
 '39--47-03',
 '40--48-03',
 '41--49-03',
 '42--50-03',
 '43--51-03',
 '44--52-03',
 '45--53-03',
 '46--54-03',
 '47--07-04',
 '48--08-04',
 '49--09-04',
 '50--10-04',
 '51--11-04',
 '52--12-04',
 '53--13-04',
 '54--14-04',
 '55--16-04',
 '56--17-04',
 '57--18-04',
 '58--19-04',
 '59--20-04',
 '60--21-04',
 '61--22-04',
 '62--23-04',
 '63--24-04',
 '64--25-04',
 '65--26-04',
 '66--28-04',
 '67--29-04',
 '68--30-04',
 '69--31-04',
 '70--32-04',
 '71--33-04',
 '72--34-04',
 

In [7]:
dir_image = dir_project+info_basic['dir_image'] + 'autopick/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)
dir_ds = dir_project+info_basic['dir_ds']
print('dir_image: ', dir_image)
print('dir_ds: ', dir_ds)

dir_image:  ./project_repartition_v3.0/output_repar_v9.1_02-01/image_v9.1_02-01/autopick/
dir_ds:  ./project_repartition_v3.0/output_repar_v9.1_02-01/ds_v9.1_02-01/


In [8]:
dir_disp =  'autopick'+info_basic['tag']+'/'
info_basic['dir_disp'] = dir_disp
dir_disp = dir_project+dir_disp
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)
print('dir_disp: ', dir_disp)

dir_disp:  ./project_repartition_v3.0/output_repar_v9.1_02-01/autopickv9.1_02-01/


In [9]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [10]:
ds_remove_all = {}
for key_subwork in key_subworks:
    print(key_subwork)
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    ds.close()
    ds_remove_all[key_subwork] = ds_remove

1--07-03
2--08-03
3--09-03
4--10-03
5--11-03
6--12-03
7--13-03
8--14-03
9--15-03
10--16-03
11--18-03
12--19-03
13--20-03
14--21-03
15--22-03
16--23-03
17--24-03
18--25-03
19--26-03
20--27-03
21--28-03
22--29-03
23--30-03
24--31-03


25--32-03
26--33-03
27--34-03
28--35-03
29--36-03
30--37-03
31--38-03
32--39-03
33--40-03
34--41-03
35--42-03
36--43-03
37--44-03
38--46-03
39--47-03
40--48-03
41--49-03
42--50-03
43--51-03
44--52-03
45--53-03
46--54-03
47--07-04
48--08-04
49--09-04
50--10-04
51--11-04
52--12-04
53--13-04
54--14-04
55--16-04
56--17-04
57--18-04
58--19-04
59--20-04
60--21-04
61--22-04
62--23-04
63--24-04
64--25-04
65--26-04
66--28-04
67--29-04
68--30-04
69--31-04
70--32-04
71--33-04
72--34-04
73--36-04
74--38-04
75--40-04
76--41-04
77--42-04
78--43-04
79--45-04
80--46-04
81--47-04
82--48-04
83--49-04
84--50-04
85--54-04
86--08-05
87--10-05
88--12-05
89--13-05
90--14-05
91--16-05
92--17-05
93--19-05
94--21-05
95--22-05
96--24-05
97--25-05
98--26-05
99--27-05
100--28-05
101--29-05
102--32-05
103--33-05
104--34-05
105--35-05
106--36-05
107--37-05
108--39-05
109--40-05
110--41-05
111--42-05
112--44-05
113--47-05
114--48-05
115--49-05
116--51-05
117--52-05
118--53-05
119--54-05
120--07-06
121--08-06
122--10-

### Pick probes' Phase velocity automatically

In [11]:
def cluster_fundamental(ds):
    global f
    global c
    global f_new
    global c_new
    global dir_ds
    global threshold
    # 归一化
    #ds = ds / np.max(ds)  
    ds = np.nan_to_num(ds)
    for i in range(ds.shape[1]):
        ds[:,i] = ds[:,i]/np.max(ds[:,i])

    binary_ds = ds > threshold
    label_ds,num_labels = measure.label(binary_ds, connectivity=2, return_num=True)
    region_areas = [np.sum(label_ds==i) for i in range(1,num_labels+1)]
    if num_labels == 0:
        return np.zeros(0),np.zeros(0)
    max_area_label = np.argmax(region_areas) + 1
    largest_cluster = label_ds == max_area_label
    indices = np.argwhere(largest_cluster)

    ff = f_new[indices[:,1]]
    cc = c_new[indices[:,0]]

    return ff,cc

In [12]:
def pick_fundamental(key):
    global f
    global c
    global f_new
    global c_new
    global flag_f
    global flag_c
    global dir_ds
    global threshold
    global dir_disp
    global dir_ds
    global info_basic
    global info_basic_bi
    global smooth_cluster
    global smooth_pick
    global key_subworks
    global ds_remove_all

    
    ds_remove = ds_remove_all[key]
    grace = 0.0

    f_start = flag_f[0]
    f_end = flag_f[1]
    c_min = flag_c[0]
    c_max = flag_c[1]

    ff_all = []
    cc_all = []


    while f_start != f_end:
        #print(f_start,f_end)
        f_new = f[np.logical_and(f>f_start,f <= f_end)]
        c_new = c[np.logical_and(c>c_min,c < c_max)]
        ds_new0 = ds_remove[np.logical_and(c>c_min,c < c_max),:]
        ds_new = ds_new0[:,np.logical_and(f>f_start,f <= f_end)]
        # 对数组进行横向的平滑
        ds_new = pd.DataFrame(ds_new)
        #ds_new = ds_new.rolling(100).mean()
        ds_new = ds_new.T
        ds_new = ds_new.rolling(smooth_cluster).mean()
        ds_new = np.array(ds_new)
        ds_new = ds_new.T
        ff,cc = cluster_fundamental(ds_new)
        if len(ff_all) == 0:
            ff_all = ff
            cc_all = cc
        elif len(ff) != 0:
            ff_all = np.hstack((ff_all,ff))
            cc_all = np.hstack((cc_all,cc))
        else:
            #print('this')
            break
        f_start = np.max(ff)
        c_max = np.max(cc[ff==f_start]) + grace
    
    # smooth
    # 转置
    ds_smooth = ds_remove.T
    ds_smooth = pd.DataFrame(ds_smooth)
    ds_smooth = ds_smooth.rolling(smooth_pick).mean()
    ds_smooth = np.array(ds_smooth)
    ds_smooth = ds_smooth.T
    ds_smooth = pd.DataFrame(ds_smooth)
    ds_smooth = ds_smooth.rolling(smooth_pick).mean()
    ds_smooth = np.array(ds_smooth)

    f_range = set(ff_all)
    f_range = list(f_range)
    c_pick = []
    for i in range(len(f_range)):
        f_this = f_range[i]
        c_this = cc_all[ff_all==f_this]
        c_range = [np.min(c_this),np.max(c_this)]
        c_new = c[np.logical_and(c>=c_range[0],c<=c_range[1])]
        c_pick.append(c_new[np.argmax(ds_smooth[np.logical_and(c>=c_range[0],c<=c_range[1]),f==f_this])])
    
    # save as txt
    np.savetxt(dir_disp+'autopick_cluster_'+key+'.txt', np.array([ff_all,cc_all]).T, fmt='%.4f', delimiter=',')
    np.savetxt(dir_disp+'autopick_'+key+'.txt', np.array([f_range,c_pick]).T, fmt='%.4f', delimiter=',')
    print('finish '+ key + ' '+str(key_subworks.index(key)+1) + '/'+str(len(key_subworks)))

In [13]:
f = info_basic_bi['f']
c = np.linspace(info_basic['c_min'],info_basic['c_max'],info_basic['c_num'])
flag_c = [0.2,1.2]
flag_f = [3,15]
threshold = 0.9
f_new = f[np.logical_and(f>flag_f[0],f < flag_f[1])]
c_new = c[np.logical_and(c>flag_c[0],c < flag_c[1])]
smooth_cluster = 30
smooth_pick = 6
flag_pick = 1

In [14]:
warnings.filterwarnings("ignore")
if flag_pick == 1:
    for key_subwork in key_subworks:
        pick_fundamental(key_subwork)

finish 1--07-03 1/657
finish 2--08-03 2/657
finish 3--09-03 3/657
finish 4--10-03 4/657
finish 5--11-03 5/657
finish 6--12-03 6/657
finish 7--13-03 7/657
finish 8--14-03 8/657
finish 9--15-03 9/657
finish 10--16-03 10/657
finish 11--18-03 11/657
finish 12--19-03 12/657
finish 13--20-03 13/657
finish 14--21-03 14/657
finish 15--22-03 15/657
finish 16--23-03 16/657
finish 17--24-03 17/657
finish 18--25-03 18/657
finish 19--26-03 19/657
finish 20--27-03 20/657
finish 21--28-03 21/657
finish 22--29-03 22/657
finish 23--30-03 23/657
finish 24--31-03 24/657
finish 25--32-03 25/657
finish 26--33-03 26/657
finish 27--34-03 27/657
finish 28--35-03 28/657
finish 29--36-03 29/657
finish 30--37-03 30/657
finish 31--38-03 31/657
finish 32--39-03 32/657
finish 33--40-03 33/657
finish 34--41-03 34/657
finish 35--42-03 35/657
finish 36--43-03 36/657
finish 37--44-03 37/657
finish 38--46-03 38/657
finish 39--47-03 39/657
finish 40--48-03 40/657
finish 41--49-03 41/657
finish 42--50-03 42/657
finish 43-

In [15]:
"""
nThreads = 60
pool = ThreadPoolExecutor(max_workers = nThreads)
for key_subwork in key_subworks:
    pool.submit(pick_fundamental,key_subwork)
pool.shutdown()
"""

'\nnThreads = 60\npool = ThreadPoolExecutor(max_workers = nThreads)\nfor key_subwork in key_subworks:\n    pool.submit(pick_fundamental,key_subwork)\npool.shutdown()\n'

In [16]:
def plot_disp(key_subwork):
    global dir_disp
    global f
    global c
    global dir_image
    global key_subworks

    fig,ax = plt.subplots(1,2,figsize=(16,6))
    dir_ds = dir_project + info_basic['dir_ds']
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]

    title0 = key_subwork
    ff_all = np.loadtxt(dir_disp+'autopick_cluster_'+key_subwork+'.txt',delimiter=',')[:,0]
    cc_all = np.loadtxt(dir_disp+'autopick_cluster_'+key_subwork+'.txt',delimiter=',')[:,1]
    f_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    c_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]

    ax[0] = plotlib.plot_fj(ax[0],ds_remove,title0,f,c,0,c_map='jet')
    ax[0].scatter(ff_all,cc_all,marker='o',s=10,color='k')
    
    ax[1] = plotlib.plot_fj(ax[1],ds_remove,title0,f,c,0,c_map='jet')
    ax[1].scatter(f_pick,c_pick,marker='o',s=10,color='k')

    plt.savefig(dir_image+'autopick_'+key_subwork+'.png',dpi=60)
    plt.tight_layout()
    plt.close()
    print('finish '+key_subwork,' ', str(key_subworks.index(key_subwork)+1),'/',str(len(key_subworks)))

In [17]:
flag_plot = 0

In [18]:
if flag_plot == 1:
    for key_subwork in key_subworks:
        plot_disp(key_subwork)

finish 1--07-03   1 / 657
finish 2--08-03   2 / 657
finish 3--09-03   3 / 657
finish 4--10-03   4 / 657
finish 5--11-03   5 / 657
finish 6--12-03   6 / 657
finish 7--13-03   7 / 657
finish 8--14-03   8 / 657
finish 9--15-03   9 / 657
finish 10--16-03   10 / 657
finish 11--18-03   11 / 657
finish 12--19-03   12 / 657
finish 13--20-03   13 / 657
finish 14--21-03   14 / 657
finish 15--22-03   15 / 657
finish 16--23-03   16 / 657
finish 17--24-03   17 / 657
finish 18--25-03   18 / 657
finish 19--26-03   19 / 657
finish 20--27-03   20 / 657
finish 21--28-03   21 / 657
finish 22--29-03   22 / 657
finish 23--30-03   23 / 657
finish 24--31-03   24 / 657
finish 25--32-03   25 / 657
finish 26--33-03   26 / 657
finish 27--34-03   27 / 657
finish 28--35-03   28 / 657
finish 29--36-03   29 / 657
finish 30--37-03   30 / 657
finish 31--38-03   31 / 657
finish 32--39-03   32 / 657
finish 33--40-03   33 / 657
finish 34--41-03   34 / 657
finish 35--42-03   35 / 657
finish 36--43-03   36 / 657
finish 37-